## Unificação das bases de dados

In [ ]:
import pandas as pd
import re
from unidecode import unidecode

In [ ]:

def normalize_municipio(nome: str) -> str:
    """Remove acentos e padroniza como 'MUNICIPIO - UF'"""
    if pd.isna(nome):
        return None
    nome = unidecode(str(nome)).upper().strip()
    m = re.search(r"\((\w{2})\)$", nome)
    uf = None
    if m:
        uf = m.group(1)
        base = nome[:m.start()].strip()
    else:
        m = re.search(r"\s-\s([A-Z]{2})$", nome)
        if m:
            uf = m.group(1)
            base = nome[:m.start()].strip()
        else:
            base = nome
    return f"{base} - {uf}" if uf else base


In [ ]:

# Carregar emendas com partidos e agregar por município
emendas = pd.read_csv('../data/emendas_por_favorecido_partidos.csv', encoding='utf-8-sig')
emendas['municipio'] = (emendas['Município Favorecido'] + ' - ' + emendas['UF Favorecido']).apply(normalize_municipio)
agg = (emendas.groupby('municipio', as_index=False)['Valor Recebido']
               .sum()
               .rename(columns={'Valor Recebido': 'valor_pix_total'}))

# Indicadores IBGE
 dens = pd.read_csv('../data/densidade.csv')
 dens['municipio'] = dens['name_muni_x'].apply(normalize_municipio)
 dens = dens[['code_muni', 'municipio', 'densidade_demografica']]

 escolar = pd.read_csv('../data/escolarizacao.csv')
 escolar['municipio'] = escolar['Territorialidades'].apply(normalize_municipio)
 escolar = escolar[['municipio', 'Taxa de Educacao 2010']]

 idhm = pd.read_csv('../data/idhm.csv')
 idhm['municipio'] = idhm['Territorialidades'].apply(normalize_municipio)
 idhm = idhm[['municipio', 'IDHM 2010']]

 pib = pd.read_csv('../data/pib_per_capita.csv')
 pib['municipio'] = pib['name_muni'].apply(normalize_municipio)
 pib = pib[['municipio', 'pib_per_capita_2021']]

 resultados = pd.read_csv('../data/resultados_eleicoes.csv')
 resultados['municipio'] = (resultados['NM_MUNICIPIO'] + ' - ' + resultados['SG_UF']).apply(normalize_municipio)
 prefeitos = resultados[resultados['DS_SIT_TOT_TURNO'] == 'ELEITO']
 prefeitos = prefeitos.drop_duplicates('municipio')[['municipio', 'NM_URNA_CANDIDATO']]                    .rename(columns={'NM_URNA_CANDIDATO': 'prefeito_eleito'})

# Merge de todas as bases
base = agg.merge(dens, on='municipio', how='left')
base = base.merge(pib, on='municipio', how='left')
base = base.merge(idhm, on='municipio', how='left')
base = base.merge(escolar, on='municipio', how='left')
base = base.merge(prefeitos, on='municipio', how='left')

# Seleciona colunas principais
dados_unificados = base[['municipio', 'code_muni', 'valor_pix_total',
                         'pib_per_capita_2021', 'densidade_demografica',
                         'IDHM 2010', 'Taxa de Educacao 2010', 'prefeito_eleito']]

dados_unificados.to_csv('../data/dados_unificados.csv', index=False, encoding='utf-8-sig')
dados_unificados.head()
